In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
def dados_tratar(df):

    #Renomear
    df.columns = nomes = [
    'srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
    'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
    'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz',
    'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt',
    'Dintpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
    'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst',
    'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
    'attack_cat', 'Label']

    # Dropar colunas não úteis
    df = df.drop(columns=['Stime', 'Ltime', 'srcip', 'dstip', 'sport', 'dsport', 'Label'])

    # Tipagem incorreta ao ler
    df['ct_ftp_cmd'] = pd.to_numeric(df['ct_ftp_cmd'], errors='coerce').fillna(0).astype(int)

    # Label Normal aparece como NaN
    df['attack_cat'] = df['attack_cat'].fillna('Normal').str.strip()

    # df['attack_cat'] possui 2 categorias (Backdoors e Backdoor) que deveriam se tornar apenas uma (Backdoor).
    df['attack_cat'].replace('Backdoors', 'Backdoor', inplace=True)

    # Identificando as top 5 categorias para 'proto'
    top_proto = df['proto'].value_counts().head(5).index.tolist()

    # Substituindo categorias não presentes nas top 5 por 'outros' para 'proto'
    df.loc[~df['proto'].isin(top_proto), 'proto'] = 'outros'

    # Identificando as top 5 categorias para 'state'
    top_state = df['state'].value_counts().head(5).index.tolist()

    # Substituindo categorias não presentes nas top 5 por 'outros' para 'state'
    df.loc[~df['state'].isin(top_state), 'state'] = 'outros'

    # Identificando as top 5 categorias para 'service'
    top_service = df['service'].value_counts().head(5).index.tolist()

    # Substituindo categorias não presentes nas top 5 por 'outros' para 'service'
    df.loc[~df['service'].isin(top_service), 'service'] = 'outros'

    # Aplicando one-hot encoding
    df_encoded = pd.get_dummies(df, columns=['proto', 'state', 'service'], dtype=int)

    # Colunas com missing ['ct_flw_http_mthd', 'is_ftp_login']
    df_final = df_encoded.fillna(0)

    return df_final


if __name__ == "__main__":
    pd.set_option('display.max_columns', None)
    print("Loading dataset...")
    #Load Data
    df1=pd.read_csv("/content/drive/MyDrive/UNSW/UNSW-NB15_1.csv", low_memory=False,header=None)
    df2=pd.read_csv("/content/drive/MyDrive/UNSW/UNSW-NB15_2.csv", low_memory=False,header=None)
    df3=pd.read_csv("/content/drive/MyDrive/UNSW/UNSW-NB15_3.csv", low_memory=False,header=None)
    df4=pd.read_csv("/content/drive/MyDrive/UNSW/UNSW-NB15_4.csv", low_memory=False,header=None)

    df = pd.concat([df1,df2,df3,df4])
    print(f'Dados importados com shape: {df.shape}')
    df  = dados_tratar(df)
    print("Distribuição do label:")
    print(df['attack_cat'].value_counts())
    print(f'Dados para AL com shape: {df.shape}')
    print("Salvar dataset")
    df.drop("attack_cat", axis=1).to_csv('/content/drive/MyDrive/UNSW/NB15_features.csv', index=False,header=False )
    df["attack_cat"].to_csv('/content/drive/MyDrive/UNSW/NB15_labels.csv', index=False,header=False )
    print("Salvar dataset... DONE")

Loading dataset...
Dados importados com shape: (2540047, 49)
Distribuição do label:
attack_cat
Normal            2218764
Generic            215481
Exploits            44525
Fuzzers             24246
DoS                 16353
Reconnaissance      13987
Analysis             2677
Backdoor             2329
Shellcode            1511
Worms                 174
Name: count, dtype: int64
Dados para AL com shape: (2540047, 57)
Salvar dataset
Salvar dataset... DONE
